## Notebook 01: Ingesta de Datos
**Objetivo de la Ingesta:** Descargar y cargar datos de contratos públicos desde la API de Datos Abiertos Colombia (SECOP II).

**Dataset:** SSECOP II - Contratos Electrónicos.)

**Fuente:** "https://www.datos.gov.co/Gastos-Gubernamentales/SECOP-II-Contratos-Electr-nicos/jbjy-vk9h"

**Actividades:**
  - Configurar SparkSession conectada al cluster
  - Descargar datos desde la API Socrata (SECOP II)
  - Cargar datos en Spark y explorar el esquema
  - Seleccionar columnas clave para ML
  - Guardar en formato Parquet optimizado
izadouet optimizadot optimizado


In [1]:
pip install sodapy


[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Importar librerías
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date
# Para control de la ingesta
from pyspark.sql.functions import current_timestamp, input_file_name
from delta import *
import os
from sodapy import Socrata
import json
import pandas as pd

## **Configurar SparkSession**

### Conectamos al Spark Master del cluster

In [3]:
master_url = "spark://spark-master:7077"

# Configuración y Añadimos Delta Lake
# Despues de master_url
builder = SparkSession.builder \
    .appName("Lab_SECOP_Bronze") \
    .master(master_url) \
    .config("spark.jars.packages", "io.delta:delta-spark_2.12:3.0.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.executor.memory", "2g") 

spark = configure_spark_with_delta_pip(builder).getOrCreate()
print(f"Spark Version: {spark.version}")
print(f"Spark Master: {spark.sparkContext.master}")

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b12cb993-1bda-4fa1-8d4d-fa093b533f0c;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.0.0 in central
	found io.delta#delta-storage;3.0.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 181ms :: artifacts dl 7ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.0.0 from central in [default]
	io.delta#delta-storage;3.0.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   |   

Spark Version: 3.5.0
Spark Master: spark://spark-master:7077


## **Descargar datos desde API Socrata**

Se seleccionaran los datos registrados en la pagina de SECOP desde el 1 de enero de 2026 para asi mismo ver que contratos fueron presentados en el primer mes de año 2026.

In [4]:
print("Extrayendo datos desde API Socrata")

client = Socrata("www.datos.gov.co", None)

query = """
SELECT *
WHERE fecha_de_firma IS NOT NULL
ORDER BY fecha_de_firma DESC
LIMIT 100000
"""

results = client.get(
    "jbjy-vk9h",
    query=query
)

print(f"Registros descargados: {len(results):,}")



Extrayendo datos desde API Socrata


26/02/07 19:03:49 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


Registros descargados: 100,000


In [5]:
df_raw = pd.DataFrame.from_records(results)
print("Columnas cargadas:")
print(df_raw.columns.tolist())

Columnas cargadas:
['nombre_entidad', 'nit_entidad', 'departamento', 'ciudad', 'localizaci_n', 'orden', 'sector', 'rama', 'entidad_centralizada', 'proceso_de_compra', 'id_contrato', 'referencia_del_contrato', 'estado_contrato', 'codigo_de_categoria_principal', 'descripcion_del_proceso', 'tipo_de_contrato', 'modalidad_de_contratacion', 'justificacion_modalidad_de', 'fecha_de_firma', 'fecha_de_fin_del_contrato', 'condiciones_de_entrega', 'tipodocproveedor', 'documento_proveedor', 'proveedor_adjudicado', 'es_grupo', 'es_pyme', 'habilita_pago_adelantado', 'liquidaci_n', 'obligaci_n_ambiental', 'obligaciones_postconsumo', 'reversion', 'origen_de_los_recursos', 'destino_gasto', 'valor_del_contrato', 'valor_de_pago_adelantado', 'valor_facturado', 'valor_pendiente_de_pago', 'valor_pagado', 'valor_amortizado', 'valor_pendiente_de', 'valor_pendiente_de_ejecucion', 'estado_bpin', 'c_digo_bpin', 'anno_bpin', 'saldo_cdp', 'saldo_vigencia', 'espostconflicto', 'dias_adicionados', 'puntos_del_acuerdo'

## **Cargar datos en Spark y explorar el esquema**

Lo correcto es que Spark lea desde disco, no recrear DataFrames desde sí mismos:


In [6]:
# Pasar a Spark
df_raw = spark.createDataFrame(df_raw)
print(f"Registros en Spark: {df_raw.count():,}")

# Guardar en formato parquet
output_path = "/opt/spark-data/raw/secop_contratos_general.parquet"

df_raw.write \
    .mode("overwrite") \
    .parquet(output_path)

print(f"Datos guardados en: {output_path}")


26/02/07 19:08:48 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
26/02/07 19:08:49 WARN TaskSetManager: Stage 0 contains a task of very large size (86725 KiB). The maximum recommended task size is 1000 KiB.


Registros en Spark: 100,000


26/02/07 19:08:53 WARN TaskSetManager: Stage 3 contains a task of very large size (86725 KiB). The maximum recommended task size is 1000 KiB.


Datos guardados en: /opt/spark-data/raw/secop_contratos_general.parquet


## **Seleccionar columnas clave para ML**

Campos clave: 
 - Referencia del Contrato
 - Precio Base
 - Departamento
 - Tipo de Contrato
 - Fecha de Firma
 - Plazo de Ejecucion
 - Proveedor Adjudicado
 - Estado del Contrato.

In [7]:
columnas_clave_ml = [
    "referencia_del_contrato",
    "valor_del_contrato",
    "departamento",
    "tipo_de_contrato",
    "fecha_de_firma",
    "duraci_n_del_contrato",
    "proveedor_adjudicado",
    "estado_contrato"
]

# Validar columnas existentes
columnas_disponibles = [c for c in columnas_clave_ml if c in df_raw.columns]

print(f"\nColumnas seleccionadas para ML ({len(columnas_disponibles)}):")
for c in columnas_disponibles:
    print(f"- {c}")

df_ml = df_raw.select(*columnas_disponibles)

print(f"\nRegistros para ML: {df_ml.count():,}")


Columnas seleccionadas para ML (8):
- referencia_del_contrato
- valor_del_contrato
- departamento
- tipo_de_contrato
- fecha_de_firma
- duraci_n_del_contrato
- proveedor_adjudicado
- estado_contrato


26/02/07 19:10:40 WARN TaskSetManager: Stage 4 contains a task of very large size (86725 KiB). The maximum recommended task size is 1000 KiB.



Registros para ML: 100,000


## **Guardar en formato Parquet optimizado**

In [8]:
output_path1 = "/opt/spark-data/raw/secop_base_ml.parquet"

print(f"Guardando datos en formato Parquet...")
print(f"Ruta: {output_path1}")

(
    df_ml.write
    .mode("overwrite")
    .parquet(output_path1)
)

print("Datos guardados exitosamente en formato Parquet")

Guardando datos en formato Parquet...
Ruta: /opt/spark-data/raw/secop_base_ml.parquet


26/02/07 19:10:54 WARN TaskSetManager: Stage 7 contains a task of very large size (86725 KiB). The maximum recommended task size is 1000 KiB.


Datos guardados exitosamente en formato Parquet


In [9]:
spark.stop()